In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [5]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

import numpy as np

print(np.mean(scores['test_score']))

{'fit_time': array([0.01254606, 0.01460266, 0.01297903, 0.01281261, 0.01232982]), 'score_time': array([0.00184608, 0.00180078, 0.00190425, 0.00179148, 0.00182056]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [6]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # 10-폴드 교차 검증
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.8574181117533719


In [10]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # 값을 바꿔가며 총 5번 실행
# n_jobs -> 병렬 실행에 사용할 CPU 코어 수 지정 (-1 : 모든 코어 사용)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(gs.best_params_) # 최적의 파라미터
print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [14]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), # 총 9개
                    'max_depth' : range(5, 20, 1), # 총 15개
                    'min_samples_split' : range(2, 100, 10)} # 총 10개

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}
0.8683865773302731


In [15]:
from scipy.stats import uniform, randint

rgen = randint(0, 10) # 정수 뽑음
rgen.rvs(10)

array([6, 9, 9, 6, 9, 3, 1, 7, 7, 6])

In [16]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103, 104, 101,  89, 103, 110,  87,  92,  93, 118]))

In [17]:
ugen = uniform(0, 1) # 실수 뽑음
ugen.rvs(10)

array([0.52607359, 0.47002802, 0.54481477, 0.59154559, 0.45130754,
       0.41942327, 0.51102809, 0.08779174, 0.33327145, 0.97078394])

In [20]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
                    'max_depth' : randint(20, 50),
                    'min_samples_split' : randint(2, 25),
                    'min_samples_leaf' : randint(1, 25)}

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 28, 'min_impurity_decrease': np.float64(0.00030000000000000003), 'min_samples_leaf': 7, 'min_samples_split': 7}
0.8681951950840305
0.8561538461538462
